In [3]:
import numpy as np
from mpl_toolkits import mplot3d
%matplotlib inline
import matplotlib.pyplot as plt
from math import sin, cos, atan, pi,e, radians as rad
from multiprocessing import Pool
from scipy import stats

!pip install progressbar2
import progressbar

In [4]:

num_atoms = 100
num_chains = 100 # 10^4
counter = [0] * 3
initial_temp=300
final_temp=700
rate = 40

In [5]:
#@title Constants
bl={}
ba=[]
ta=[]
ep={}
epo={}
Ua=np.matrix([])
Ub=np.matrix([])

def initializeConstants():
  global bl 
  bl= {
      "si-o": 164,
      "si-c": 190
  }

  global ba
  ba= [143, 110]

  global ta 
  ta= [180, 60, -60]

  global ep 
  ep= {
      "sigma": 0.286,
      "sigma`": 0.286,
      "delta": 0.06
  }

  global epo
  epo= {
      "sigma": 1,
      "sigma`": 1,
      "delta": 1
  }
  global Ua
  Ua= np.matrix([
      [1, ep['sigma'], ep['sigma']],
      [1, ep['sigma`'], 0],
      [1, 0, ep['sigma`']]
  ])

  global Ub
  Ub= np.matrix([
      [1, ep['sigma'], ep['sigma']],
      [1, ep['sigma'], ep['delta']],
      [1, ep['delta'], ep['sigma`']]
  ])
    
initializeConstants()

In [4]:
u1 = np.matrix("1, 0, 0")
un = np.matrix("1; 1; 1")
mp = np.linalg.matrix_power

zn = (u1 * mp(Ua * Ub, int((num_atoms-2)/2)) * un)[0, 0]
print(zn)
  
def getPn(i, prev_state, req_state = -1):
  pn = 0
  n = num_atoms
  
  def get_u_dash(u_dash):
    if req_state != -1:
      val = u_dash[prev_state, req_state]
      u_dash.fill(0)
      u_dash[prev_state, req_state] = val
    else:
      val = u_dash[:, prev_state].copy()
      u_dash.fill(0)
      u_dash[:, prev_state] = val
    return u_dash
  
  if i % 2 == 0:
    pn = (u1 * mp(Ua * Ub, int((i-2)/2)) * get_u_dash(Ua.copy()) * Ub * mp(Ua * Ub, int((n-i-2)/2)) * un)[0, 0]
  else:
    pn = (u1 * mp(Ua * Ub, int((i-3)/2)) * Ua * get_u_dash(Ub.copy()) * mp(Ua * Ub, int((n-i-1)/2)) * un)[0, 0]
    
  return pn/zn

def getNextState(i, prev_state):
  from random import random
  rand = random()

  p_ep_t_i = getPn(i, prev_state, 0) # req_state = Trans
  p_ep_i1 = getPn(i-1, prev_state)
  q_t = p_ep_t_i / p_ep_i1
    
  if rand < q_t:
    # print("Next predicted state: Trans")
    return 0 # Trans
  p_ep_g_plus_i = getPn(i, prev_state, 1) # req_state = Gauche+
  q_g_plus = p_ep_g_plus_i / p_ep_i1
    
  if rand > q_t and rand < (q_t + q_g_plus):
   #  print("Next predicted state: Gauche+")
    return 1 # Gauche+
  else:
    # print("Next predicted state: Gauche-")
    return 2 # Gauche-

8.632580008221922e+16


In [6]:

def calc_cordinates(p1, p2, p3, new_state, ba):
  inv = np.linalg.inv
  
  t = np.matrix([
      [1, 0, 0, -p2[0]],
      [0, 1, 0, -p2[1]],
      [0, 0, 1, -p2[2]],
      [0, 0, 0, 1,]
  ])
  
  p4 = p3-p2
  p5 = np.sqrt(sum(p4**2))
  a, b, c = p4/p5
  d = np.sqrt(b**2+c**2)
  
  if d != 0:
    cost = c/d
    sint = b/d

    rx = np.matrix([
        [1, 0, 0, 0],
        [0, cost, -sint, 0],
        [0, sint, cost, 0],
        [0, 0 ,0 ,1]
    ])

  ry = np.matrix([
      [d, 0, -a, 0],
      [0, 1, 0, 0],
      [a, 0, d, 0],
      [0, 0, 0, 1],
  ])
  
  p1 = np.append(p1, 1)
  trans_p1 = np.matrix(p1).T
  if d != 0:
    trans_p1 = (ry * rx * t * trans_p1)
  else:
    trans_p1 = (ry * t * trans_p1)
  
  angle = atan(trans_p1[0]/trans_p1[1])
  
  if trans_p1[1] < 0:
    angle = pi - angle
    
  rz = np.matrix([
      [cos(angle), -sin(angle),	0,	0,],
      [sin(angle), cos(angle), 0, 0],
      [0, 0, 1,	0],
      [0,	0, 0, 1]
  ])
  
  trans_p1 = rz * trans_p1
  
  l = bl['si-o']  
  x = abs(l * cos(rad(ba-90)) * sin(rad(ta[new_state])))
  y = abs(l * cos(rad(ba-90)) * cos(rad(ta[new_state])))
  z = l + abs(l * cos(rad(180-ba)))

  if trans_p1[1] < 0:
    if new_state == 1:
      y = -y
    elif new_state == 2:
      x = -x
      y = -y
  else:
    if new_state == 1:
      x = -x
    elif new_state == 0:
      y = -y
    newp = np.matrix([[x], [y], [z], [1]])
  if d != 0:
    newp = inv(t) * inv(rx) * inv(ry) * inv(rz) * newp
  else:
    newp = inv(t) * inv(ry) * inv(rz) * newp

  return np.squeeze(np.asarray(newp))[:3]

In [ ]:
def generateChains(nc, na, i):
    r = []
    s = []
    cn = [1, 0, 0]
    l = bl['si-o']
    p1_ = np.array([0, 0, 0])
    p2_ = np.array([0, 0, l])
    p3_ = np.array([0, l*sin(rad(37)), (l*cos(rad(37))) + l])
    for j in progressbar.progressbar(range(nc)):
#     for j in range(nc):
        p1 = p1_
        p2 = p2_
        p3 = p3_
        next_state = 0
        chain = [p1, p2, p3]
        for i in range(3, na):
            newp = calc_cordinates(p1, p2, p3, next_state, ba[i%2])
            chain.append(newp)
            p1 = p2
            p2 = p3
            p3 = newp
            next_state = getNextState(i, next_state)
            cn[next_state] += 1
        cm = np.average(chain, axis=0)
        s.append(np.sqrt(np.sum((chain - cm)**2)/num_atoms))
        r.append(np.sqrt(sum(chain[-1]**2)))
    return (r, s, cn)
r = []
s = []
num_process = 2
p = Pool(num_process)
async_results = []
for i in range(num_process):
    async_results.append(p.apply_async(generateChains, (int(num_chains/num_process), num_atoms, i)))
for res in async_results:
    part_r, part_s, cn = res.get()
    r += part_r
    s += part_s
    print(cn)
    counter = list(map(sum, zip(counter,cn)))


In [ ]:

print("Average Radius of Gyration: %f pm" % np.average(s))

n_bins = 'auto'
fig, ax = plt.subplots(figsize=(8, 4))
n, bins, patches = ax.hist(s, n_bins, histtype='step')
ax.set_xlabel('Radius of Gyration, s (pm)')
ax.set_ylabel('Number of polymer chains')
ax.xaxis.label.set_size(15)
ax.yaxis.label.set_size(15)
plt.show()